In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L3\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['AT.csv', 'BE.csv', 'BG.csv', 'CH.csv', 'CZ.csv', 'DK.csv', 'ES.csv', 'FR.csv', 'GR.csv', 'IT.csv', 'NL.csv', 'PT.csv', 'SI.csv', 'SK.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[12]))

,Demand,Normalized_net,Temperature,DNI,DHI
0,0.00,0.017762,-11.261,0.0,0.0
1,1045.47,0.445247,-10.975,0.0,0.0
2,1004.79,0.428613,-10.772,0.0,0.0
3,983.79,0.420027,-10.667,0.0,0.0
4,998.67,0.426111,-10.467,0.0,0.0
...,...,...,...,...,...
43819,1367.13,0.576772,0.053,0.0,0.0
43820,1261.40,0.533539,-0.207,0.0,0.0
43821,1192.50,0.505367,-0.203,0.0,0.0
43822,1133.98,0.481438,-0.032,0.0,0.0


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[12]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temperature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_SI,Temp_SI
0,0.00,-11.261
1,1045.47,-10.975
2,1004.79,-10.772
3,983.79,-10.667
4,998.67,-10.467
...,...,...
43819,1367.13,0.053
43820,1261.40,-0.207
43821,1192.50,-0.203
43822,1133.98,-0.032


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_SI,Temp_SI,Lag-2,Lag-7
0,1318.90,-2.913,1214.31,0.00
1,1261.35,-3.012,1193.23,1045.47
2,1297.72,-2.931,1201.24,1004.79
3,1274.97,-2.727,1202.41,983.79
4,1414.87,-2.633,1327.59,998.67
...,...,...,...,...
43651,1367.13,0.053,1436.34,1256.83
43652,1261.40,-0.207,1359.71,1202.55
43653,1192.50,-0.203,1270.75,1155.58
43654,1133.98,-0.032,1156.78,1074.09


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 38s 22ms/step - loss: 0.0166 - mape: 11545.1904 - mae: 0.1046 - val_loss: 0.0170 - val_mape: 22.3490 - val_mae: 0.1055
Epoch 2/100
1105/1105 [==============================] - 34s 31ms/step - loss: 0.0091 - mape: 11393.5674 - mae: 0.0742 - val_loss: 0.0115 - val_mape: 18.2621 - val_mae: 0.0847
Epoch 3/100
1105/1105 [==============================] - 34s 31ms/step - loss: 0.0069 - mape: 10252.5137 - mae: 0.0629 - val_loss: 0.0185 - val_mape: 22.3636 - val_mae: 0.1115
Epoch 4/100
1105/1105 [==============================] - 35s 32ms/step - loss: 0.0062 - mape: 9847.0586 - mae: 0.0585 - val_loss: 0.0170 - val_mape: 22.5052 - val_mae: 0.1100
Epoch 5/100
1105/1105 [==============================] - 34s 31ms/step - loss: 0.0057 - mape: 9867.7852 - mae: 0.0557 - val_loss: 0.0215 - val_mape: 25.1432 - val_mae: 0.1255
Epoch 6/100
1105/1105 [==============================] - 34s 31ms/step - loss: 0.0054 - mape: 9596.1143 - mae: 0.0539 - va

1105/1105 [==============================] - 31s 28ms/step - loss: 0.0023 - mape: 9507.7090 - mae: 0.0347 - val_loss: 0.0277 - val_mape: 28.3482 - val_mae: 0.1428
Epoch 48/100
1105/1105 [==============================] - 35s 31ms/step - loss: 0.0022 - mape: 9241.8672 - mae: 0.0344 - val_loss: 0.0259 - val_mape: 27.2367 - val_mae: 0.1369
Epoch 49/100
1105/1105 [==============================] - 35s 31ms/step - loss: 0.0023 - mape: 9365.3906 - mae: 0.0351 - val_loss: 0.0265 - val_mape: 26.9575 - val_mae: 0.1353
Epoch 50/100
1105/1105 [==============================] - 34s 31ms/step - loss: 0.0022 - mape: 9453.0195 - mae: 0.0341 - val_loss: 0.0243 - val_mape: 25.9050 - val_mae: 0.1319
Epoch 51/100
1105/1105 [==============================] - 34s 31ms/step - loss: 0.0021 - mape: 9226.0430 - mae: 0.0337 - val_loss: 0.0246 - val_mape: 27.2043 - val_mae: 0.1328
Epoch 52/100
1105/1105 [==============================] - 35s 31ms/step - loss: 0.0021 - mape: 9350.8076 - mae: 0.0333 - val_loss: 0.

1105/1105 [==============================] - 29s 26ms/step - loss: 0.0012 - mape: 8984.2334 - mae: 0.0260 - val_loss: 0.0228 - val_mape: 26.9533 - val_mae: 0.1300
Epoch 94/100
1105/1105 [==============================] - 29s 26ms/step - loss: 0.0013 - mape: 8908.3516 - mae: 0.0270 - val_loss: 0.0254 - val_mape: 26.6603 - val_mae: 0.1330
Epoch 95/100
1105/1105 [==============================] - 29s 26ms/step - loss: 0.0012 - mape: 8986.3340 - mae: 0.0262 - val_loss: 0.0204 - val_mape: 24.1533 - val_mae: 0.1197
Epoch 96/100
1105/1105 [==============================] - 29s 26ms/step - loss: 0.0012 - mape: 9017.8809 - mae: 0.0255 - val_loss: 0.0223 - val_mape: 25.0466 - val_mae: 0.1244
Epoch 97/100
1105/1105 [==============================] - 29s 26ms/step - loss: 0.0012 - mape: 8929.2305 - mae: 0.0256 - val_loss: 0.0206 - val_mape: 24.4516 - val_mae: 0.1189
Epoch 98/100
1105/1105 [==============================] - 29s 26ms/step - loss: 0.0011 - mape: 8966.2754 - mae: 0.0254 - val_loss: 0.

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 3343.1898629665375


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 7ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.27845496, 0.24700826, 0.23968314, ..., 0.3392128 , 0.29750717,
       0.270818  ], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.37099958, 0.385915  , 0.40530228, ..., 0.33297492, 0.3007069 ,
       0.27462008])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,1261.46,1093.624756
1,1288.51,1036.594238
2,1323.67,1023.309753
3,1348.93,1034.205200
4,1582.86,1128.434082
...,...,...
4363,1367.13,1385.406982
4364,1261.40,1287.860352
4365,1192.50,1203.812866
4366,1133.98,1128.177124


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

8.345574684755794